In [1]:
#%
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import glob
import os
import cartopy.crs as ccrs
from scipy.optimize import least_squares
import warnings

In [2]:
#%% calculate seasonality index SI and time-lag phi
# input:
    # daily timeseries of p and ep
# output:
    # seasonality index of precipitation si [0]
    # time lag between max monthly p and ep phi [1]
    
def calculate_si_phi(p,ep):
    # find start and end dates (we want to start 01-01 and end 31-12)
    for j in p.index:
        if(j.month==1 and j.day==1):
            start_date = j
            break
    for j in p.index:
        if(j.month==12 and j.day==31):
            end_date = j
            
    p_annual = p.loc[start_date:end_date].groupby(pd.Grouper(freq="Y")).sum()
    pa = p_annual.mean()
    p_monthly = p.loc[start_date:end_date].groupby(pd.Grouper(freq="M")).sum()
    pm = p_monthly.groupby([p_monthly.index.month]).mean()
    ep_annual = ep.loc[start_date:end_date].groupby(pd.Grouper(freq="Y")).sum()
    epa = ep_annual.mean()
    ep_monthly = ep.loc[start_date:end_date].groupby(pd.Grouper(freq="M")).sum()
    epm = ep_monthly.groupby([p_monthly.index.month]).mean()
    
    # SEASONALITY INDEX P 
    a = np.zeros(12)
    for k in range(12):
        a[k] = np.abs(pm[k + 1]-(pa/12))
    sip = (1/pa)*np.sum(a)
    
    # SEASONALITY INDEX EP 
    a = np.zeros(12)
    for k in range(12):
        a[k] = np.abs(epm[k + 1]-(epa/12))
    siep = (1/epa)*np.sum(a)
    
    # TIMING
    epm_max_month = epm.idxmax()
    pm_max_month = pm.idxmax()
    phi = np.abs(epm_max_month - pm_max_month)
    if(phi>6):
        phi = 12 + min(epm_max_month,pm_max_month) - max(epm_max_month,pm_max_month)
        
    return(sip,siep,phi)

In [3]:
#%% seasonality timing index (van stijn)

# input:
    # dataframe with daily P EP 
# output: 
    # seasonality timing index dpstar

class ST_index_calc_stijn:
    def __init__(self, df):
        self.df = df.copy()
    
    def get_dp_star(self):
        dp, sp = self._least_squares_optimization('P')
        de, se = self._least_squares_optimization('EP')
        # print(dp)
        return dp * np.sign(de) * np.cos(2 * np.pi * (sp - se)/365)
    
    def _least_squares_optimization(self, variable):
        pars = least_squares(self._get_difference_estimate_reality, (1, 0), args=[variable]).x
        return pars
    
    def _get_difference_estimate_reality(self, pars, variable):
        P_est = self._get_daily_estimates(pars, variable)
        return self.df[variable] - P_est
    
    def _get_daily_estimates(self, pars, var):
        dp, sp = pars
        s = self.df[var]
        P_mean = s.mean()
        t = s.index.dayofyear.values
        if dp > 0:
            cr = -0.001 * dp**4 + 0.026 * dp ** 3 - 0.245 * dp ** 2 + 0.2432 * dp - 0.038
        else:
            cr = 0
        addition = (1 + cr + dp * np.sin(2*np.pi*(t - sp) / 365))
        P_est = P_mean * addition
        P_est[P_est < 0] = 0
        return P_est

In [ ]:
# make dataframe with catchment characteristics

# no warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    a = np.genfromtxt('/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/catch_id_selected_lowercase_wb.txt',dtype='str')
    fol='/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/'
    df_cc = pd.DataFrame(index=a, columns=['p_mean_gswp','ep_mean_gleam','q_mean','t_mean_gswp','ai','rc','ea_wb','si_p','si_ep','phi','st','tc','ntc','nonveg','start_year','end_year'])
    d = pd.read_csv(f'{fol}/p_q_ep_timeseries_selected_catchments/mean_q_p_ep.csv', index_col=0)
    t = pd.read_csv(f'{fol}/gsim_shapes_treecover.csv', index_col=0)

    for i in range (len(a)):
        b = a[i]

        # daily timesteries p and ep
        l = glob.glob(f'{fol}p_ep_timeseries_selected_catchments/daily/{b}*.csv')
        pep = pd.read_csv(l[0], index_col=0)
        pep.index = pd.to_datetime(pep.index)

        # long term mean
        df_cc.loc[b].p_mean_gswp = d.loc[b].p
        df_cc.loc[b].ep_mean_gleam = d.loc[b].ep
        df_cc.loc[b].q_mean = d.loc[b].q
        df_cc.loc[b].start_year = int(d.loc[b].y_start)
        df_cc.loc[b].end_year = int(d.loc[b].y_end)

        df_cc.loc[b].ai = d.loc[b].ep / d.loc[b].p
        df_cc.loc[b].rc = d.loc[b].q / d.loc[b].p
        df_cc.loc[b].ea_wb = d.loc[b].p - d.loc[b].q

        # get si and phi from daily p ep timeseries
        df_cc.loc[b].si_p = calculate_si_phi(pep.p, pep.ep)[0]
        df_cc.loc[b].si_ep = calculate_si_phi(pep.p, pep.ep)[1]
        df_cc.loc[b].phi = calculate_si_phi(pep.p, pep.ep)[2]

        # # get st index
        # df_st = pd.DataFrame(index=pep.index,columns=['P','EP']) # need to give df with columns P and EP as input
        # df_st[['P','EP']] = pep[['p','ep']]
        # k = ST_index_calc_stijn(df_st)
        # stindex = k.get_dp_star()
        # df_cc.loc[b,('st')] = stindex

        # tree cover
        df_cc.loc[b, ('tc')] = t.loc[b, ('mean_tc')]
        df_cc.loc[b, ('ntc')] = t.loc[b, ('mean_ntc')]
        df_cc.loc[b, ('nonveg')] = t.loc[b, ('mean_nonveg')]

    df_cc.to_csv(f'{fol}/catchment_characteristics.csv')

In [ ]:
df_cc